In [1]:
pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.1.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import joblib

In [3]:
df = pd.read_csv("../data/recipes.csv")

In [4]:
df.drop(columns=["Unnamed: 0", "prep_time", "cook_time", "total_time", "servings",
    "yield", "rating", "url", "nutrition", "img_src"], inplace = True)

In [5]:
df = df[['ingredients', 'directions','timing','cuisine_path', 'recipe_name']].dropna().drop_duplicates()

In [6]:
cuisine_counts = df['cuisine_path'].value_counts()
threshold = 4
df['cuisine_grouped'] = df['cuisine_path'].apply(lambda x: x if cuisine_counts[x] > threshold else 'Other')

In [7]:
df['combined_text'] = df['ingredients'] + ' ' + df['directions'] + ' ' + df['timing']

In [8]:
import re
def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)  # remove punctuation
    text = re.sub(r'\d+', '', text)  # remove numbers
    return text

df['combined_text'] = df['combined_text'].apply(clean_text)


In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=1000, ngram_range=(1, 2), stop_words='english')
X = vectorizer.fit_transform(df['combined_text'])

In [10]:
y_cuisine = df["cuisine_grouped"]
y_recipe = df["recipe_name"]

In [33]:
# X_train, X_test, y_train_c, y_test_c, y_train_r, y_test_r = train_test_split(
#     X, y_cuisine, y_recipe, test_size=0.2, random_state=42, stratify=y_cuisine
# )


In [34]:
X_train_c, X_test_c, y_train_c, y_test_c = train_test_split(X, y_cuisine, test_size=0.2, random_state=42)
X_train_r, X_test_r, y_train_r, y_test_r = train_test_split(X, y_recipe, test_size=0.2, random_state=42)

In [15]:
cuisine_model = RandomForestClassifier()
cuisine_model.fit(X_train_c, y_train_c)

recipe_model = RandomForestClassifier()
recipe_model.fit(X_train_r, y_train_r)

RandomForestClassifier()

In [16]:
pip install catboost

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.1.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [17]:
from catboost import CatBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

In [18]:
cat_model = CatBoostClassifier(iterations=100, verbose=50, allow_writing_files=False)
cat_model.fit(X_train_c, y_train_c)

Learning rate set to 0.5
0:	learn: 3.3016937	total: 1.9s	remaining: 3m 8s
50:	learn: 0.7979357	total: 1m 29s	remaining: 1m 26s
99:	learn: 0.4505796	total: 2m 56s	remaining: 0us


In [21]:
print(X_train_c.shape)
print(y_train_c.shape)
print(X_train_r.shape)
print(y_train_r.shape)


(768, 1000)
(768,)
(768, 1000)
(768,)


In [22]:
cat_model = CatBoostClassifier(iterations=100, verbose=10, allow_writing_files=False)
cat_model.fit(X_train_r, y_train_r)


Learning rate set to 0.5
0:	learn: 6.5198589	total: 27.4s	remaining: 45m 13s
10:	learn: 5.4564293	total: 5m 12s	remaining: 42m 9s
20:	learn: 4.8753438	total: 9m 45s	remaining: 36m 44s
30:	learn: 4.8357312	total: 14m 29s	remaining: 32m 15s
40:	learn: 4.8025267	total: 18m 57s	remaining: 27m 16s
50:	learn: 4.7016973	total: 23m 28s	remaining: 22m 33s
60:	learn: 4.6573688	total: 28m 9s	remaining: 18m
70:	learn: 4.6203995	total: 32m 49s	remaining: 13m 24s
80:	learn: 4.5861495	total: 37m 9s	remaining: 8m 43s
90:	learn: 4.5540957	total: 41m 46s	remaining: 4m 7s
99:	learn: 4.5256720	total: 45m 59s	remaining: 0us


In [24]:
cat_model = CatBoostClassifier(iterations=50,  allow_writing_files=False)
cat_model.fit(X_train_r, y_train_r)

Learning rate set to 0.5
0:	learn: 6.5198589	total: 26.9s	remaining: 22m
1:	learn: 6.4455660	total: 54.8s	remaining: 21m 56s
2:	learn: 6.3107137	total: 1m 25s	remaining: 22m 23s
3:	learn: 6.2073314	total: 1m 56s	remaining: 22m 15s
4:	learn: 6.1388046	total: 2m 24s	remaining: 21m 40s
5:	learn: 6.0526754	total: 2m 51s	remaining: 20m 57s
6:	learn: 5.9150378	total: 3m 18s	remaining: 20m 20s
7:	learn: 5.7875519	total: 3m 45s	remaining: 19m 45s
8:	learn: 5.6407285	total: 4m 13s	remaining: 19m 13s
9:	learn: 5.5602221	total: 4m 39s	remaining: 18m 38s
10:	learn: 5.4564293	total: 5m 6s	remaining: 18m 5s
11:	learn: 5.3663936	total: 5m 32s	remaining: 17m 34s
12:	learn: 5.3036871	total: 5m 59s	remaining: 17m 3s
13:	learn: 5.1835498	total: 6m 26s	remaining: 16m 33s
14:	learn: 5.0679736	total: 6m 53s	remaining: 16m 4s
15:	learn: 4.9830006	total: 7m 20s	remaining: 15m 35s
16:	learn: 4.9343144	total: 7m 46s	remaining: 15m 5s
17:	learn: 4.8929809	total: 8m 12s	remaining: 14m 36s
18:	learn: 4.8877143	tot

In [35]:
svm_linear = SVC(kernel='linear')
svm_linear.fit(X_train_c, y_train_c)

SVC(kernel='linear')

In [36]:
svm_linear = SVC(kernel='linear')
svm_linear.fit(X_train_r, y_train_r)

SVC(kernel='linear')

In [37]:
svm_rbf = SVC(kernel='rbf')
svm_rbf.fit(X_train_c, y_train_c)

SVC()

In [38]:
svm_rbf = SVC(kernel='rbf')
svm_rbf.fit(X_train_r, y_train_r)

SVC()

In [39]:
log_reg = LogisticRegression(max_iter=1000)
log_reg.fit(X_train_c, y_train_c)

LogisticRegression(max_iter=1000)

In [43]:
log_reg = LogisticRegression(max_iter=1000)
log_reg.fit(X_train_r, y_train_r)

LogisticRegression(max_iter=1000)

In [44]:
print("y_test_c sample:", y_test_c[:5])
print("y_pred sample:", y_pred[:5])


y_test_c sample: 440                          /Salad/Green Salad Recipes/
478    /Side Dish/Sauces and Condiments/Canning and P...
447                            /Cuisine/European/French/
889                  /Desserts/Cakes/Cheesecake Recipes/
372      /Desserts/Fruit Desserts/Peach Dessert Recipes/
Name: cuisine_grouped, dtype: object
y_pred sample: ['Hawaiian Cole Slaw' 'BC Cherry Jam' 'Pear Butter' 'Persimmon Cheesecake'
 'Cherry Pound Cake']


In [45]:
models = {
    "CatBoost": cat_model,
    "Linear SVM": svm_linear,
    "RBF SVM": svm_rbf,
    "Logistic Regression": log_reg
}

for name, model in models.items():
    y_pred = model.predict(X_test_c)
    acc = accuracy_score(y_test_c, y_pred)
    print(f"{name} Accuracy (Cuisine Prediction): {acc:.2f}")

CatBoost Accuracy (Cuisine Prediction): 0.00
Linear SVM Accuracy (Cuisine Prediction): 0.00
RBF SVM Accuracy (Cuisine Prediction): 0.00
Logistic Regression Accuracy (Cuisine Prediction): 0.00
